In [ ]:
import pandas as pd
import random
import math
import sys

In [ ]:
# a matrix defining dependencies between the Pokemon types used in the game
# rows correspond to attacking types (as provided in the comments below)
# columns correspond to defending Pokemon types in the same order as the attacking types
type_matrix = [
    # grass, fire, water, fighting, ground, electric
    [0.5, 0.5, 2.0, 1.0, 2.0, 1.0], # grass move
    [2.0, 0.5, 0.5, 1.0, 1.0, 1.0], # fire move
    [0.5, 2.0, 0.5, 1.0, 2.0, 1.0], # water move
    [1.0, 1.0, 1.0, 1.0, 1.0, 1.0], # fighting move
    [0.5, 2.0, 1.0, 1.0, 1.0, 2.0], # ground move
    [0.5, 1.0, 2.0, 1.0, 0.0, 0.5] # electric move
]

# convert the matrix to pandas dataframe to add labels
type_matrix_dataframe = pd.DataFrame(type_matrix)
type_matrix_dataframe.columns = ["Grass", "Fire", "Water", "Fighting", "Ground", "Electric"]
type_matrix_dataframe.index = ["Grass", "Fire", "Water", "Fighting", "Ground", "Electric"]

In [ ]:
class Move():
    """A class representing a Pokemon move"""
    
    def __init__(self, name, m_type, category, power, accuracy, effect):
        """Initialize attributes"""
        self.name = name.title()
        self.m_type = m_type.title() # type of the move, e.g. fire
        self.category = category.title() # category of the move, e.g. physical
        self.power = power # power of the move
        self.accuracy = accuracy # accuracy of the move, between 1 and 100
        self.effect = effect # short description of move's effect
        
    def display_info(self):
        """Display move info"""
        print("{0}: \n\tType: {1}\tCategory: {2}".format(self.name, self.m_type, self.category))
        if self.name == "Grass Knot" or self.name == "Earthquake":
            print("\tPower: 40-100\tAccuracy: {0}\n\tEffect: {1}".format(self.accuracy, self.effect))
        elif self.effect == "Inflicts status":
            print("\tPower: NA\tAccuracy: {0}\n\tEffect: {1}".format(self.accuracy, self.effect))
        elif self.category == "Other":
            print("\tPower: NA\tAccuracy: NA\n\tEffect: {0}".format(self.effect))
        else:
            print("\tPower: {0}\tAccuracy: {1}\n\tEffect: {2}".format(self.power, self.accuracy, self.effect))
        print()

In [ ]:
class Pokemon():
    """A class representing a Pokemon"""
        
    def __init__(self, name, p_type, stats, *args, **kwargs):
        """Initialize attributes"""
        self.name = name.title()
        self.p_type = p_type.title() # type of the Pokemon, all Pokemons are monotype in the current version of the game
        # Pokemon stats to be used during battles, with a spread provided by randint to simulate IV/EV
        # each stat holds 2 values: current value (which can change during battle) and base value
        # changes of HP are permanent, other stats go back to the base level after withdrawing
        self.HP = [random.randint(stats[0]-10, stats[0]+10)] * 2
        self.attack = [random.randint(stats[1]-10, stats[1]+10)] * 2
        self.defence = [random.randint(stats[2]-10, stats[2]+10)] * 2
        self.sp_attack = [random.randint(stats[3]-10, stats[3]+10)] * 2
        self.sp_defence = [random.randint(stats[4]-10, stats[4]+10)] * 2
        self.speed = [random.randint(stats[5]-10, stats[5]+10)] * 2
        self.moves = kwargs.get('moves', []) # stores moves Pokemon can use during battles
        self.status = kwargs.get('status', False) # used to track if pokemon has a HP-removing status
        self.fainted = kwargs.get('fainted', False) # used to track if pokemon fainted
        
    def display_info(self):
        """Display Pokemon info"""
        print("{0}: \n\tType: {1}\n\tHP: {2}/{3}".format(self.name, self.p_type, self.HP[0], self.HP[1]), end = "\t\t")
        print("Attack: {0}\n\tDefence: {1}\t\tSp. Attack: {2}".format(self.attack[1], self.defence[1], self.sp_attack[1]))
        if self.sp_defence[1] >= 100: # ensure correct formatting of output
            print("\tSp. Defence: {0}\tSpeed: {1}\n\tMoves:".format(self.sp_defence[1], self.speed[1]), end = "")
        else:
            print("\tSp. Defence: {0}\t\tSpeed: {1}\n\tMoves:".format(self.sp_defence[1], self.speed[1]), end = "")
        self.display_moves(True)
            
    def teach_moves(self, new_moves):
        """Teach Pokemon its moveset"""
        self.moves = list(new_moves)
    
    def display_moves(self, is_basic):
        """Display Pokemon moveset"""
        if is_basic: # only display move names
            for move in self.moves:
                if len(move.name) > 7: # ensure correct formatting of output
                    print("\t{0}".format(move.name),end = "")
                else:
                    print("\t{0}".format(move.name),end = "\t")
            print("\n")
        else: # display all move info
            for i, move in enumerate(self.moves):
                print("[{0}]".format(i+1), end=" ")
                move.display_info()

In [ ]:
class Trainer():
    """A class representing a Pokemon trainer with 3 Pokemons"""
    
    def __init__(self,name):
        """Initiate attributes"""
        self.name = name.title()
        self.pokemons = [] # stores the Pokemons the trainer is battling with (starts with 3)
        
    def display_pokemons(self):
        """Display info on all Pokemons the trainer has"""
        for i, pokemon in enumerate(self.pokemons):
            print("[{0}]".format(i+1), end=" ")
            pokemon.display_info()
        
    def add_pokemon(self,pokemon):
        """Add a Pokemon to trainer's team when they choose it"""
        self.pokemons.append(pokemon) # no checks regarding the number of Pokemons the trainer has are necessary in the game
        
    def remove_pokemon(self,pokemon):
        """Remove a pokemon from trainer's team when it faints"""
        self.pokemons.remove(pokemon)

In [ ]:
class Battle():
    """A class simulating a Pokemon battle"""
    
    def __init__(self, trainer, pokemon, battle_number):
        """Initialize attributes"""
        self.trainer = trainer # the player
        self.wild_pokemon = pokemon # randomly initiated wild Pokemon
        self.number = battle_number # stores information on how many battles have been fought during the Pokemon journey
        self.battling_pokemon = None # battling Pokemon is chosen after the battle starts
        self.is_over = False # bool controlling if the battle finished or no
        print("\n\nBattle no. {0}:".format(self.number))
        print("A wild {0} has been encountered!".format(self.wild_pokemon.name)) # start a Pokemon battle
        
    def choose_pokemon(self):
        """Choose a Pokemon to send out either at the beginning of the batttle or when switching Pokemons"""
        print("\nYour Pokemons:")
        self.trainer.display_pokemons()
        # if trainer has more than 1 Pokemon left, let them choose the battling Pokemon
        if len(self.trainer.pokemons) > 1:
            while True: # ensure valid user input
                try:
                    choice = int(input("Which Pokemon do you wish to send out to battle? (Press the corresponding number) "))
                    if choice > len(self.trainer.pokemons) or choice < 1:
                        print("Not a valid input\n")
                    elif self.trainer.pokemons[choice-1] == self.battling_pokemon:
                        print("Cannot send out a Pokemon you just withdrew!\n".format(self.battling_pokemon.name))
                    else:
                        print("{0} has been chosen!".format(self.trainer.pokemons[choice-1].name))
                        break
                except ValueError: 
                    print("Not a valid input.\n")
            self.battling_pokemon = self.trainer.pokemons[choice-1]
        # if only 1 Pokemon left, send it out to battle
        else:
            print("You only have one Pokemon left, so {0} will be sent out to battle.\n".format(self.trainer.pokemons[0].name))
            self.battling_pokemon = self.trainer.pokemons[0]
        
    def which_pokemon(self, is_trainers):
        """Return trainer's / wild Pokemon based on a boolean value"""
        if is_trainers:
            pokemon = self.battling_pokemon
        else:
            pokemon = self.wild_pokemon
        return pokemon
    
    def draw_move(self):
        """Draw one of the 4 moves the wild pokemon can use at random"""
        # return tuple containing the move and a bool stating it is not trainer's Pokemon
        return (random.choice(self.wild_pokemon.moves), False)
    
    def choose_move(self):
        print("\n{0}'s moves:".format(self.battling_pokemon.name))
        self.battling_pokemon.display_moves(False) # the boolean value prompts the function to provide the more detailed display
        while True: # ensure valid user input
            try:
                choice = int(input("Which move should {0} use? (Press 1, 2, 3 or 4) ".format(self.battling_pokemon.name)))
                if choice in (1, 2, 3, 4):
                    break
                else:
                    print("Not a valid input.\n")
            except ValueError:
                print("Not a valid input.\n")
        # return tuple containing the move and a bool stating it is trainer's Pokemon
        return (self.battling_pokemon.moves[choice-1], True)  
    
    # damage_tuple contains the amount of damage and information which Pokemon is affected         
    def alter_remaining_HP(self, damage_tuple, is_status): # is_status bool controls if it's status damage
        """Remove or add health points to the current HP value of a Pokemon"""
        pokemon = self.which_pokemon(damage_tuple[1]) # determine which Pokemon is affected based on a boolean value
        wild_string = "" if damage_tuple[1] else "The wild " # a string ensuring clearer output
        pokemon.HP[0] -= damage_tuple[0] # negative damage if healing
        if pokemon.HP[0] <= 0: # if HP falls below zero
            damage = damage_tuple[0] + pokemon.HP[0] # used to display how much damage the Pokemon took
            pokemon.HP[0] = 0
            pokemon.fainted = True # faints if HP falls to zero
        elif pokemon.HP[0] > pokemon.HP[1]: # can't have more HP than its initial HP value
            damage = damage_tuple[0] + pokemon.HP[0] - pokemon.HP[1] 
            pokemon.HP[0] = pokemon.HP[1]
        else:
            damage = damage_tuple[0]
        if is_status:
            print("\n{0}{1} was hit by the status condition!".format(wild_string, pokemon.name))
            print("{0}{1} lost {2} HP!".format(wild_string, pokemon.name, damage))
        else:
            if damage_tuple[0] > 0:
                print("{0}{1} lost {2} HP!".format(wild_string, pokemon.name, damage))
            elif damage_tuple[0] < 0:
                print("{0}{1} gained {2} HP!".format(wild_string, pokemon.name, -damage))
        print("{0}{1}'s remaining HP: {2}/{3}".format(wild_string, pokemon.name, pokemon.HP[0], pokemon.HP[1]))
        
    def heal(self, is_trainers):
        """Increase remaining HP of a Pokemon after a healing move is used"""
        pokemon = self.which_pokemon(is_trainers) # determine which Pokemon used the move
        healed_HP = int(-pokemon.HP[1]/2) # healed HP is half of the initial HP of a Pokemon
        # return tuple containing healed HP and a bool stating if it is trainer's Pokemon
        return (healed_HP, is_trainers)
    
    def inflict_status(self, is_trainers):
        """Inflict HP-removing status"""
        pokemon = self.which_pokemon(is_trainers) # determine which Pokemon will get a status condition
        wild_string = "" if is_trainers else "The wild " # a string ensuring clearer output
        if pokemon.status:
            print("{0}{1} already had a status condition!".format(wild_string, pokemon.name))
        else:
            pokemon.status = True
            print("{0}{1} got a status condition!".format(wild_string, pokemon.name))
        
    def increase_stat(self, is_trainers, effect): # effect determines which stat is raised
        """Increase one of Pokemon's stats (attack, defence, sp. attack, sp.defence, speed)"""
        pokemon = self.which_pokemon(is_trainers) # determine which Pokemon's stat should be raised
        wild_string = "" if is_trainers else "The wild " # a string ensuring clearer output
        # dictionary mapping effect strings to corresponding attribute names
        effect_map = {
            "Increases attack": "attack",
            "Increases defence": "defence",
            "Increases sp. attack": "sp_attack",
            "Increases sp. defence": "sp_defence",
            "Increases speed": "speed"
        }
        if effect in effect_map:
            stat_name = effect_map[effect]
            stat_value = getattr(pokemon, stat_name) # a list with 2 elements (current and initial stat values)
            # a decision was made that speed will be increased faster than other stats
            stat_increase = 0.5 * stat_value[1] if stat_name != "speed" else stat_value[1]
            stat_value[0] += stat_increase # alter the current value of the stat
            if "_" in stat_name: # ensure correct output
                stat_name = stat_name.replace("_", ". ")
            print("{0}{1}'s {2} increased!".format(wild_string, pokemon.name, stat_name))
        else:
            print("Invalid effect string: {0}\n".format(effect))
    
    def use_move(self, move_tuple): # move_tuple contains the move and boolean information which Pokemon used it
        """Enforce effects of the chosen move"""
        pokemon = self.which_pokemon(move_tuple[1]) # attacking Pokemon
        wild_string = "" if move_tuple[1] else "The wild " # a string ensuring clearer output
        print("\n{0}{1} used {2}!".format(wild_string, pokemon.name, move_tuple[0].name))
        required_accuracy = random.randint(1, 100) # draw a number representing the accuracy of the move required to hit
        # check if the move hits
        if move_tuple[0].accuracy >= required_accuracy:
        # perform relevant actions based on move's category and effect if it does hit
            if move_tuple[0].category == "Other"and move_tuple[0].effect == "Heals user":
                self.alter_remaining_HP(self.heal(move_tuple[1]), False) # heal the attacking Pokemon
            elif move_tuple[0].category == "Other" and move_tuple[0].effect == "Inflicts status":
                self.inflict_status(not move_tuple[1]) # inflict a status condition on the defending (hence the "not") Pokemon
            elif move_tuple[0].category == "Other":
                self.increase_stat(move_tuple[1], move_tuple[0].effect) # increase one of the attacking Pokemon's stats
            else:
                self.alter_remaining_HP(self.deal_damage(move_tuple), False) # increase the attacking Pokemon's stats
                # the "not" is passed from the deal_damage function
        else:
            print("The attack missed!")
    
    def deal_damage(self, move_tuple): # move_tuple contains the move and boolean information which Pokemon used it
        """Deal damage on the defending Pokemon as a result of a move"""
        attacking_pokemon = self.which_pokemon(move_tuple[1])
        defending_pokemon = self.which_pokemon(not move_tuple[1])
        move = move_tuple[0] # used by the attacking pokemon
        # determine the factor resulting from Pokemon types dependencies
        type_factor = type_matrix_dataframe.at[move.m_type,defending_pokemon.p_type]
        # determine the effectiveness of the move based on the type factor
        if type_factor == 2:
            print("It's super effective!")
        elif type_factor == 0.5:
            print("It's not very effective...")
        elif type_factor == 0:
            print("It didn't affect the opposing Pokemon...")
        # calculate the power of the move
        move_power = move.power * type_factor
        # get STAB (same type attack boost)
        if move.m_type == attacking_pokemon.p_type:
            move_power *= 1.5
        # calculate the damage based on move power and Pokemons' stats
        if move.category == "Physical":
            # formula invented for the needs of the program, possibly requires tuning
            damage = int(move_power * 0.5 * math.sqrt(attacking_pokemon.attack[0] / defending_pokemon.defence[0]))
        else:
            damage = int(move_power * 0.5 * math.sqrt(attacking_pokemon.sp_attack[0] / defending_pokemon.sp_defence[0]))
        # return tuple containing removed HP and a bool stating which Pokemon had the HP removed
        return (damage, not move_tuple[1]) 
    
    def deal_status_damage(self, is_trainers):
        """Deal damage on a Pokemon as a result of status condition"""
        pokemon = self.which_pokemon(is_trainers) # determine which Pokemon gets affected by the status condition
        status_damage = int(pokemon.HP[1]/8) # status removes 1/8th of Pokemon's initial HP
        # return tuple containing HP removed due to the status and a bool stating if it is trainer's Pokemon
        return (status_damage, is_trainers)
    
    def reset_stats(self):
        """Bring the stats back to their base level"""
        self.battling_pokemon.attack[0] = self.battling_pokemon.attack[1]
        self.battling_pokemon.defence[0] = self.battling_pokemon.defence[1]
        self.battling_pokemon.sp_attack[0] = self.battling_pokemon.sp_attack[1]
        self.battling_pokemon.sp_defence[0] = self.battling_pokemon.sp_defence[1]
        self.battling_pokemon.speed[0] = self.battling_pokemon.speed[1]
        
    def withdraw_pokemon(self, is_forced): # is_forced is a bool determining if Pokemon fainted (true) or not (false)
        """Withdraw battling Pokemon either on switch-out or when it faints"""
        self.reset_stats() # reset stat boosts on switch-out 
        if is_forced: # if fainting is the reason for withdrawing 
            print("\n{0} fainted!".format(self.battling_pokemon.name))
            self.trainer.remove_pokemon(self.battling_pokemon) # remove fainted Pokemon from trainer's party
        print("\n{0}, come back!\n".format(self.battling_pokemon.name))
        if is_forced and len(self.trainer.pokemons) == 0:
            self.loose_battle() # loose the battle if out of Pokemons
        return is_forced
            
    def win_battle(self):
        """Set the status of the battle to over after winning"""
        print("\nThe wild {0} fainted!".format(self.wild_pokemon.name))
        self.is_over = True
        
    def loose_battle(self):
        """Set the status of the battle to over after loosing"""
        print("\nYou're out of Pokemons! You lose!")
        self.is_over = True
 
    def perform_round(self):
        """Perform all the actions which occur in a round of a Pokemon battle"""
        pokemons_in_order = [] # used to determine the order in which Pokemons use moves and get hit by status condition
        priority = random.choice(["trainers","wild"]) # quantity to randomly determine who goes first if speeds equal
        is_trainers_fainted = False # determines if trainer's Pokemon has fainted this round
        # 1 represent using a move, while 2 represents switching the battling Pokemon
        
        if len(self.trainer.pokemons) == 1: # if one Pokemon left, cannot switch
            choice = "1"
        else:
            while True: # ensure valid user input
                choice = input("What would you like to do?\n\t-use a move [1]\n\t-switch Pokemons [2]\n\t")
                if choice in ["1", "2"]:
                    break
                else:
                    print("Not a valid input.")
                    
        if choice == "1": # using a move
            battling_move_tuple = self.choose_move() # the move used by trainer's Pokemon and bool stating it is trainer's
            # bool controlling whether trainer's Pokemon moves before the wild Pokemon
            is_trainers_faster = self.battling_pokemon.speed > self.wild_pokemon.speed\
            or self.battling_pokemon.speed == self.wild_pokemon.speed and priority == "trainers"
            pokemons_in_order = [self.battling_pokemon, self.wild_pokemon] if is_trainers_faster\
            else [self.wild_pokemon, self.battling_pokemon]
            # iterating through defeding Pokemons instead of attacking to simplify accessing them to check for fainting
            for defending_pokemon in reversed(pokemons_in_order): # slower Pokemon defends first
                move_tuple = battling_move_tuple if defending_pokemon == self.wild_pokemon else self.draw_move()
                self.use_move(move_tuple) # attacking Pokemon uses its move
                if defending_pokemon.fainted:
                    if defending_pokemon == self.battling_pokemon: # if the wild Pokemon was attacking
                        # the function returns it's only parameter (True)
                        is_trainers_fainted = self.withdraw_pokemon(True) # bool states that the withdrawal is forced
                        if self.is_over: # if the battle is over
                            return
                        break
                    else: # if trainer's Pokemon was attacking
                        self.win_battle() # player wins if the wild Pokemon faints
                        return
        else: # switching Pokemons
            # the function returns it's only parameter (False)
            is_trainers_fainted = self.withdraw_pokemon(False) # bool states that the withdrawal is voluntary
            self.choose_pokemon()
            self.use_move(self.draw_move()) # the wild Pokemon uses its move after the switch
            if self.battling_pokemon.fainted: # if the Pokemon which just been sent out faints from the hit
                # the function returns it's only parameter (True)
                is_trainers_fainted = self.withdraw_pokemon(True) # bool states that the withdrawal is forced
                if self.is_over: # if the battle is over
                    return
                self.choose_pokemon() # need to send out another Pokemon
                
        # the moves were used, now the Pokemons are checked for status conditions
        if is_trainers_fainted: # if only the wild Pokemon on the battle field after moves were used
            if self.wild_pokemon.status:
                # self.deal_status_damage takes a bool controlling if the affected Pokemon is trainer's (False)
                # self.alter_remaining_HP takes a damage tuple and a bool controlling if it's status damage (True)
                self.alter_remaining_HP(self.deal_status_damage(False), True)
                if self.wild_pokemon.fainted: # if wild Pokemon faints due to status condition
                    self.win_battle() # player wins if the wild Pokemon faints
                    return
            self.choose_pokemon()
        else:
            # the order has to be determined again as the trainer's Pokemon might have been switched out
            is_trainers_faster = self.battling_pokemon.speed > self.wild_pokemon.speed\
            or self.battling_pokemon.speed == self.wild_pokemon.speed and priority == "trainers"
            pokemons_in_order = [(self.battling_pokemon), (self.wild_pokemon)] if is_trainers_faster\
            else [(self.wild_pokemon), (self.battling_pokemon)]
            # iterate through Pokemons on the battlefield to check if they have status condition
            for pokemon in pokemons_in_order:
                if pokemon.status:
                    self.alter_remaining_HP(self.deal_status_damage(pokemon == self.battling_pokemon), True)
                    if pokemon.fainted:
                        if pokemon == self.battling_pokemon: # if trainer's Pokemon fainted
                            # the function returns it's only parameter (True)
                            is_trainers_fainted = self.withdraw_pokemon(True) # bool states that the withdrawal is forced
                            if self.is_over: # if the battle is over
                                return
                            self.choose_pokemon()
                        else: # if the wild Pokemon fainted
                            self.win_battle() # player wins if the wild Pokemon faints
                            return       

In [ ]:
# initiating several instances of the Move class to define all the moves available in the current version of the game
Grassknot = Move("Grass Knot", "Grass", "Special", random.randint(40 ,100), 100, "Deals damage")
Solarbeam = Move("Solarbeam", "Grass", "Special", 120, 50, "Deals damage")
Synthesis = Move("Synthesis", "Grass", "Other", 0, 100, "Heals user")
Growth = Move("Growth", "Grass", "Other", 0, 100, "Increases sp. attack")
Earthquake = Move("Earthquake", "Ground", "Physical", random.randint(40 ,100), 100, "Deals damage")
Heatwave = Move("Heat Wave", "Fire", "Special", 80, 80, "Deals damage")
Poisonpowder = Move("Poison Powder", "Grass", "Other", 0, 80, "Inflicts status")
Withdraw = Move("Withdraw", "Water", "Other", 0, 100, "Increases sp. defence")
Slam = Move("Slam", "Fighting", "Physical", 75, 100, "Deals damage")
Willowisp = Move("Will-o-Wisp", "Fire", "Other", 0, 80, "Inflicts status")
Megapunch = Move("Mega Punch", "Fighting", "Physical", 90, 85, "Deals damage")
Flamethrower = Move("Flamethrower", "Fire", "Special", 95, 70, "Deals damage")
Wildcharge = Move("Wildcharge", "Electric", "Physical", 110, 70, "Deals damage")
Firepunch = Move("Fire Punch", "Fire", "Physical", 75, 100, "Deals damage")
Agility = Move("Agility", "Electric", "Other", 0 ,100, "Increases speed")
Bulkup = Move("Bulk Up", "Fighting", "Other", 0, 100, "Increases attack")
Waterpulse = Move("Water Pulse", "Water", "Special", 60, 100, "Deals damage")
Aquaring = Move("Aqua Ring", "Water", "Other", 0, 100, "Heals user")
Thunderbolt = Move("Thunderbolt", "Electric", "Special", 95, 70, "Deals damage")
Thunderpunch = Move("Thunderpunch", "Electric", "Physical", 75, 100, "Deals damage")
Harden = Move("Harden", "Ground", "Other", 0, 100, "Increases defence")
Charge = Move("Charge", "Electric", "Other", 0, 100, "Heals user")

In [ ]:
# a dictionary contatining names of Pokemons as keys and tuples of corresponding types, stats and moves as values
pokemons = {
    "Venusaur": ("Grass", [100, 70, 100, 140, 120, 70], [Grassknot, Synthesis, Growth, Poisonpowder]),
    "Tropius": ("Grass", [100, 70, 100, 140, 120, 70], [Solarbeam, Growth, Heatwave, Slam]),
    "Torterra": ("Grass", [100, 70, 100, 140, 120, 70], [Solarbeam, Synthesis, Earthquake, Withdraw]),
    "Charizard": ("Fire", [100, 110, 70, 120, 90, 110], [Solarbeam, Heatwave, Willowisp, Megapunch]),
    "Arcanine": ("Fire", [100, 110, 70, 120, 90, 110], [Willowisp, Flamethrower, Wildcharge, Agility]),
    "Magmortar": ("Fire", [100, 110, 70, 120, 90, 110], [Willowisp, Megapunch, Firepunch, Bulkup]),
    "Blastoise": ("Water", [100, 80, 120, 90, 130, 80], [Earthquake, Withdraw, Megapunch, Waterpulse]),
    "Lapras": ("Water", [100, 80, 120, 90, 130, 80], [Withdraw, Waterpulse, Aquaring, Thunderbolt]),
    "Ludicolo": ("Water", [100, 80, 120, 90, 130, 80], [Grassknot, Growth, Waterpulse, Aquaring]),
    "Hitmonchan": ("Fighting", [100, 160, 80, 60, 80, 120], [Megapunch, Firepunch, Agility, Thunderpunch]),
    "Lucario": ("Fighting", [100, 160, 80, 60, 80, 120], [Megapunch, Agility, Waterpulse, Earthquake]),
    "Toxicroak": ("Fighting", [100, 160, 80, 60, 80, 120], [Grassknot, Poisonpowder, Slam, Bulkup]),
    "Golem": ("Ground", [100, 130, 130, 80, 110, 50], [Earthquake, Slam, Firepunch, Harden]),
    "Flygon": ("Ground", [100, 130, 130, 80, 110, 50], [Solarbeam, Earthquake, Heatwave, Agility]),
    "Gastrodon": ("Ground", [100, 130, 130, 80, 110, 50], [Earthquake, Poisonpowder, Waterpulse, Harden]),
    "Lanturn": ("Electric", [100, 80, 80, 130, 80, 130], [Slam, Waterpulse, Aquaring, Thunderbolt]),
    "Manectric": ("Electric", [100, 80, 80, 130, 80, 130], [Slam, Flamethrower, Wildcharge, Charge]),
    "Electivire": ("Electric", [100, 80, 80, 130, 80, 130], [Megapunch, Bulkup, Thunderpunch, Charge])
}

In [ ]:
# helper functions

def display_pokemons():
    """Display all Pokemons available in the game"""
    for pokemon, characteristics in pokemons.items():
        if len(pokemon) > 6: # ensure correct formatting of output
            print("{0}:\t{1} type".format(pokemon, characteristics[0]))
        else:
            print("{0}:\t\t{1} type".format(pokemon, characteristics[0]))
        print("Moves:", end = "")
        for move in characteristics[2]:
            if len(move.name) > 7: # ensure correct formatting of output
                print("\t{0}".format(move.name), end = "")
            else:
                print("\t{0}".format(move.name), end = "\t")
        print("\n")    

def choose_pokemons():
    """State the names of 3 Pokemons you want in your party"""
    names = [] # list to collect the names
    while len(names) < 3: # until you provide 3 valid names
        name = input("Which pokemon do you choose? ").title()
        if name in pokemons.keys(): # check with the list of available Pokemons
            if name not in names: # check if you already have that Pokemon chosen, you can't have 2 of the same species
                names.append(name) # add the name to the list of names
            else:
                print("You cannot have 2 Pokemons of the same species.")
        else:
            print("No such Pokemon exists in this game.")
    print("\nGreat, let's start your Pokemon journey!")
    return names

def draw_pokemon():
    """Draw a Pokemon name at random from the list of available Pokemons"""
    return random.choice(list(pokemons))

def initiate_pokemon(name):
    """Initiate a Pokemon and then teach it 4 moves"""
    try:
        pokemon = Pokemon(name, pokemons[name][0], pokemons[name][1])
        pokemon.teach_moves(pokemons[name][2])
    except:
        print("Failed to initiate a Pokemon!")
        print("The simulator will stop now.")
        sys.exit()
    return pokemon

In [ ]:
# main

# display welcome messages
print("Hello, welcome to the Pokemon battle simulator!\n")
trainer_name = input("What is your name?\t")
trainer = Trainer(trainer_name) # initialize a Pokemon trainer 
print("\nFirst you have to choose 3 Pokemons that will accompany you on your journey.")
print("Here are the Pokemons you can choose from. You can only pick one Pokemon from each species.\n")

# display available Pokemons
display_pokemons()

# choose your Pokemons
party = choose_pokemons() # already contains correctly formatted names (first upper case, then lower case)
for name in party:
    trainer.add_pokemon(initiate_pokemon(name))
    
# start battling
all_fainted = False # checks if all of trainer's Pokemons fainted
started_battles = 0 # counter variable
while not all_fainted:
    started_battles += 1
    # initiate the wild Pokemon randomly from the list of available Pokemons
    wild_pokemon = initiate_pokemon(random.choice(list(pokemons)))
    battle = Battle(trainer, wild_pokemon, started_battles)
    battle.choose_pokemon() # choose the Pokemon to start the battle with
    while not battle.is_over:
        battle.perform_round()
    if len(trainer.pokemons) == 0: # if out of Pokemons
        all_fainted = True
        
# summarize the results of the game
print("This is the end of your journey. I hope you enjoyed it!")
if started_battles == 2: # if won battles == 1
    print("During your journey you won 1 battle.")
else:
    print("During your journey you won {0} battles.".format(started_battles - 1))